# Analysis on yahoo adjustmet close prices

## Initialization
The project initialization

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import matplotlib.pyplot as plt
%matplotlib inline

### Set paths to data folders

In [2]:
dir_data_raw = os.path.join('..', '..', 'data_raw')
dir_data_processing = os.path.join('..', '..', 'data_processing')

### Set dates

In [3]:
date_start = dt.datetime.strptime('2006-01-01', '%Y-%m-%d')
date_end = dt.datetime.strptime('2021-06-30', '%Y-%m-%d') 

### Get tickers DF

_set a path to the tickers.csv file._

In [4]:
tickers_file = os.path.join(dir_data_processing, 'tickers', 'all_tickers.csv')

In [5]:
tickers_df = pd.read_csv(tickers_file)

In [6]:
tickers_df

,ticker,ticker_index,ticker_yahoo,ticker_quandl
0,AAL,AAL_FTSE,AAL.L,LSE/AAL
1,ABF,ABF_FTSE,ABF.L,LSE/ABF
2,ACA,ACA_CAC,ACA.PA,EURONEXT/ACA
3,AC,AC_CAC,AC.PA,EURONEXT/AC
4,ADM,ADM_FTSE,ADM.L,LSE/ADM
...,...,...,...,...
162,WG,WG_FTSE,WG.L,LSE/WG
163,WMT,WMT_DJIA,WMT,XNYS/WMT
164,WPP,WPP_FTSE,WPP.L,LSE/WPP
165,WTB,WTB_FTSE,WTB.L,LSE/WTB


### Set thresholds

In [7]:
max_daily_ratio = 1.55
min_daily_ratio = 0.55

In [8]:
bad_prices = 0
for index, t in tickers_df.iterrows():
    price_file = os.path.join(dir_data_raw, 'prices', 'ready', "%s.csv" % t['ticker_index'])
    price_df = pd.read_csv(price_file)
    
    #check last and first date
    df_start = dt.datetime.strptime(price_df['Date'].iloc[0], '%Y-%m-%d')
    if df_start > date_start:
        print("Short series for %s starts at %s" % (t['ticker_index'], day['Date']))
    
    df_end = dt.datetime.strptime(price_df['Date'].iloc[-1], '%Y-%m-%d')    
    if df_end < date_end:
        print("Short series for %s ends at %s" % (t['ticker_index'], day['Date']))
        
    prev_day = None
    price_ratios = [1]
    show_flag = False
    for i, day in price_df.iterrows():
        if prev_day is None:
            prev_day = day
            continue
        day_ratio = day['Adj Close'] / prev_day['Adj Close'] 
        if day_ratio > max_daily_ratio or day_ratio < min_daily_ratio:
            show_flag = True
            print("anomalies %s for %s at %s" % (day_ratio, t['ticker_index'], day['Date']))
        price_ratios.append(day_ratio) 
        prev_day = day

            
    price_df['Ratio'] = pd.Series(price_ratios).values
    if show_flag:
        bad_prices += 1
        price_df.plot.line(x='Date', y='Ratio', title=t['ticker_index'])
        plt.show()
        
print("Total %s noisy series" % bad_prices)        

Total 0 noisy series
